In [508]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow.keras import layers
import time
from tensorflow import data
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense,ZeroPadding2D, BatchNormalization, Activation, Layer, ReLU, LeakyReLU,Conv2D,AveragePooling2D,UpSampling2D,Reshape,Flatten
from tensorflow.keras.models import Model


In [509]:
BATCH_SIZE = 1024


In [510]:
data = np.load('tcPred.npy')

In [511]:
train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(60000).batch(1024)


In [512]:
data.shape

(21263, 147)

In [513]:
DSHAPE = 147

In [514]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))

    model.add(layers.Dense(DSHAPE, activation='relu', use_bias=False, input_shape=(DSHAPE,)))


    

    return model

In [515]:

noise = tf.random.normal([1, DSHAPE])
generated_image = generator(noise, training=False)


In [516]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))
    model.add(layers.BatchNormalization())

    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))
   
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [517]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()


In [518]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

In [519]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [520]:
generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

In [521]:
checkpoint_dir = './training_checkpoints3'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [522]:
EPOCHS = 50
noise_dim = DSHAPE
num_examples_to_generate = 50

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [523]:
train_dataset

<BatchDataset element_spec=TensorSpec(shape=(None, 147), dtype=tf.float32, name=None)>

In [524]:
def train_step(images, gen_losses, disc_losses):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)

        disc_loss = discriminator_loss(real_output, fake_output)
        gen_losses = gen_losses.append(gen_loss.numpy())
        disc_losses = disc_losses.append(disc_loss.numpy())
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [525]:

def train(dataset, epochs, gen_losses, disc_losses, gloss, dloss):
  for epoch in range(epochs):
    start = time.time()


    for image_batch in dataset:
        train_step(image_batch, gen_losses,disc_losses)
    print("gen_loss =" + str(gen_losses[-1]))
    print("disc_loss =" + str(disc_losses[-1]))
    gloss.append(gen_losses[-1])
    dloss.append(disc_losses[-1])

    # Produce images for the GIF as you go
    x = generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  saved = generate_and_save_images(generator,
                           epochs,
                           seed)
  return saved

In [526]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)
  return predictions


In [527]:
gen_losses = []
disc_losses = []
gloss = []
dloss = []

In [528]:
discriminator = make_discriminator_model()
generator = make_generator_model()

In [529]:
final = train(train_dataset,EPOCHS, gen_losses, disc_losses, gloss, dloss)


gen_loss =0.74790424
disc_loss =1.3901504
Time for epoch 1 is 1.3260841369628906 sec
gen_loss =0.76873827
disc_loss =1.3094921
Time for epoch 2 is 1.2360258102416992 sec
gen_loss =0.85500413
disc_loss =1.1627905
Time for epoch 3 is 1.233963966369629 sec
gen_loss =0.97593963
disc_loss =0.9474943
Time for epoch 4 is 1.227733850479126 sec
gen_loss =1.2233144
disc_loss =0.7494606
Time for epoch 5 is 1.216874122619629 sec
gen_loss =1.5193243
disc_loss =0.5610739
Time for epoch 6 is 1.340061902999878 sec
gen_loss =1.8429918
disc_loss =0.4419149
Time for epoch 7 is 1.2238600254058838 sec
gen_loss =2.1377354
disc_loss =0.4231651
Time for epoch 8 is 1.5244152545928955 sec
gen_loss =2.3765304
disc_loss =0.31952667
Time for epoch 9 is 1.3262419700622559 sec
gen_loss =2.583664
disc_loss =0.30382168
Time for epoch 10 is 1.1973788738250732 sec
gen_loss =2.7924395
disc_loss =0.24832076
Time for epoch 11 is 1.1975138187408447 sec
gen_loss =2.941852
disc_loss =0.22931208
Time for epoch 12 is 1.20117712

In [530]:
final.eval

<bound method _EagerTensorBase.eval of <tf.Tensor: shape=(50, 147), dtype=float32, numpy=
array([[0.35870025, 0.13616791, 0.52748597, ..., 0.        , 0.        ,
        0.        ],
       [0.51423615, 0.08325513, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.19759572, 0.5894429 , 0.16179591, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.3432797 , 0.24079943, 0.3046231 , ..., 0.        , 0.        ,
        0.        ],
       [0.6641716 , 0.39121783, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31480506, 0.36993173, 0.09988488, ..., 0.        , 0.05688674,
        0.        ]], dtype=float32)>>

In [531]:
generated_images = generator(tf.random.normal([1000, noise_dim]))


In [501]:
generated_images

<tf.Tensor: shape=(100, 147), dtype=float32, numpy=
array([[1.2872279 , 1.146098  , 1.3402513 , ..., 0.        , 0.        ,
        0.        ],
       [1.1493005 , 0.8231228 , 0.8965121 , ..., 0.        , 0.        ,
        0.        ],
       [0.06037267, 0.14508954, 0.8527348 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.1334085 , 1.412725  , 1.1848545 , ..., 0.        , 0.        ,
        0.        ],
       [0.93419856, 1.3574717 , 0.5026217 , ..., 0.        , 0.        ,
        0.        ],
       [0.73384696, 0.3946967 , 0.78128475, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [502]:
col = np.load('gitCol.npy', allow_pickle=True)

In [503]:
col.shape

(147,)

In [504]:
generated = pd.DataFrame(generated_images, columns=col)

In [505]:
save = generated.to_pickle("gen_data3.pkl")


In [506]:
len(gen_losses)

1470

In [507]:

plt.figure(figsize=(10,8))
plt.plot(list(range(0,EPOCHS)),gloss)
plt.plot(list(range(0,EPOCHS)),dloss)
plt.title('Model Loss over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['generator loss','discriminator loss'])
plt.savefig('Learning_curve gitdnn')
plt.show
plt.close()